In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers GPUtil numba rouge bertviz

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 3.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 55.3 MB/s eta 0:00:0000:010:01
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=de6976078192ec6413253f6ddf64934eedc1880b9bde6a55c83a5a9a707dbbf4
  Stored in directory: /root/.cache/pip/wheels/b1/e7/99/2b32600270cf23194c9860f029d3d5db075f250bc39028c045
Successfully built GPUtil
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.119 which is incompatible.


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import re
import os
import csv
import copy
import math
import time
import datetime
import tqdm
import logging
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import glob
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Dropout
import inspect
from collections import OrderedDict
from torch.nn import LayerNorm
from torch.nn.init import xavier_uniform_
from GPUtil import showUtilization as gpu_usage
from numba import cuda as cuda1
from torch.utils.data.distributed import DistributedSampler, Sampler
from torch.nn.modules.container import ModuleList
from torch.optim.lr_scheduler import *
from torch.optim import *
from bertviz import model_view, head_view
from rouge import Rouge
from torch.nn.modules.transformer import _get_activation_fn, _get_clones
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.modules import TransformerDecoderLayer as ImportedTransformerDecoderLayer
from transformers import *

In [25]:
# data_dir = '/home/msc2/Project/Sanskrit_Data'
# save_dir = '/home/msc2/Project/Sanskrit_Data/Models/'
# model_path = '/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/'
# checkpoint_path = '/media/msc2/A6982E8F982E5DD5/SEM3/Checkpoint/checkpoint_27.pt'

data_dir = f'/kaggle/input/pointer-generator-sanskrit/'
save_dir = f'/kaggle/working/'
model_path = '/kaggle/input/pointer-generator-sanskrit/sans_robert/sans_robert'
checkpoint_path = f'/kaggle/input/tensors/checkpoint_27.pt'

device  = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
model = RobertaModel.from_pretrained(model_path, output_attentions = True)
tokenizer = RobertaTokenizer.from_pretrained(model_path)
checkpoint = torch.load(checkpoint_path, map_location = 'cpu')

loading configuration file /kaggle/input/pointer-generator-sanskrit/sans_robert/sans_robert/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_attentions": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /kaggle/input/pointer-generator-sanskrit/sans_robert/sans_robert/pytorch_model.bin
Some weights of the model checkpoint at /kaggle/input/pointer-generator-sanskrit/sans_robert/sans_r

In [7]:
def fix_state_dict(checkpoint):
    new_state_dict = OrderedDict()
    for k, v in checkpoint.items():
        name = k.replace("module.", "")
        new_state_dict[name] = v
    return new_state_dict
model.load_state_dict(fix_state_dict(checkpoint), strict=False);

In [8]:
#model.resize_token_embeddings(len(tokenizer))

In [9]:
def gpu_mem(string = ''):
    if torch.cuda.is_available():
        print(string)
        gpu_usage()

def get_cuda_status():
    if torch.cuda.is_available():
        print('Device count: ', torch.cuda.device_count())
        print('Current device: ',device)

def free_gpu_cache(stats = False):
    if torch.cuda.is_available():
        if stats:    
            print('\nInitial GPU usage')
            gpu_usage()
        torch.cuda.empty_cache()
        cuda1.select_device(0)
        cuda1.close()
        cuda1.select_device(0)
        if stats:
            print('\nGPU usage after emptying the cache')
            gpu_usage()

def _get_subsequent_mask(src_sz, tgt_sz):
    mask = (torch.triu(torch.ones(src_sz, tgt_sz)) == 1).transpose(0,1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask==1, float(0.0))
    return mask

def _get_square_subsequent_mask(sz):
    return _get_subsequent_mask(sz, sz)

def Embed(num_embeddings, embedding_dim, padding_idx):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx = padding_idx)
    nn.init.normal_(m.weight, mean = 0, std = embedding_dim**(-.5))
    nn.init.constant_(m.weight[padding_idx], 0)
    return m

def padding_trg(trg_ids, trg_ground_ids, trg_key_padding_mask, max_len):
    '''This function helps in the padding of the target data
    Returns : target_ids, target_real_values (for calculating loss), target_mask.'''
    trg_len = len(trg_ids)
    trg_mask = [1]*trg_len
    trg_padding = [1]*(max_len - trg_len)
    trg_mask_padding = [0]*(max_len - trg_len)
    trg_ids += trg_padding
    trg_ground_ids+= trg_padding
    trg_mask+= trg_mask_padding
    return trg_ids, trg_ground_ids, trg_mask

class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, dropout = 0.1, max_seq_len = 512):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p = dropout)
        pe = torch.zeros(max_seq_len, embedding_dim)
        position = torch.arange(0, max_seq_len, dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * (-math.log(10000.0) / embedding_dim))
        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)
        pe = pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
def delete_data_tensors():
    os.remove(data_dir+'/train.pt')
    os.remove(data_dir+'/dev.pt')
    os.remove(data_dir+'/test.pt')
    
def clear_dir(dirname):
    for f in glob.glob(dirname+'*'):
        os.remove(f)
    print('Success!')

def print_model_status(model):
    print(next(model.parameters()).is_cuda)

def get_clones(module, layers):
    module_list = [copy.deepcopy(module).to(device) for i in range(layers)]
    #gpu_mem()
    return ModuleList(module_list)

def retrieve_name(var):
    for fi in reversed(inspect.stack()):
        names = [var_name for var_name, var_val in fi.frame.f_locals.items() if var_val is var]
        if len(names) > 0:
            return names[0]

def printf(var, shape = True):
    if shape == True:
        print(f'Size of {retrieve_name(var)} : {var.shape}')
        return
    print(f'\n{retrieve_name(var)} : {var}\n----------------------------------------------------------------------------------------------------------------')
    return

def decode_text(var, back = False):
    if back == True:
        return tokenizer.decode(var)
    print(f'\n{retrieve_name(var)} : {tokenizer.decode(var)}\n----------------------------------------------------------------------------------------------------------------')
    return

def decode(var, back = False):
    outs = []
    if back == True:
        for i in range(var.shape[0]):
            outs.append(decode_text(var[i], back = True))
        return outs
    for i in range(var.shape[0]):
        decode_text(var[i])
    return

def write_one(data, file_path):
    with open(file_path, 'a+') as f:
        f.write(data)
        f.write('\n--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n')
    f.close()
    return

def write_to_file(data, file_path):
    outs = decode(data, back = True)
    for out in outs:
        write_one(out, file_path)
    return

free_gpu_cache(stats = True)

In [10]:
# clear_dir(save_dir)
#os.remove('/kaggle/working/predictions.txt')

In [11]:
class Decoder(nn.Module):
    def __init__(self, decoder_layer, decoder_final_layer, num_layers, norm = None):
        super(Decoder, self).__init__()
        decoder_layer = decoder_layer
        self.layers = get_clones(decoder_layer , num_layers-1)
        self.final_layer = decoder_final_layer
        self.num_layers = num_layers
        self.norm = norm
        
    def forward(self, tgt, mem, tgt_mask = None, mem_mask = None, tgtkey_pad_mask = None, memkey_pad_mask = None):
        output = tgt
        for i in range(self.num_layers-1):
            output = self.layers[i](output, mem,
                                    tgt_mask = tgt_mask, memory_mask = mem_mask,
                                    tgt_key_padding_mask = tgtkey_pad_mask,
                                    memory_key_padding_mask = memkey_pad_mask,
                                    
                                   )
        # output and mem of the shapes [Seq, Batch_size, Embedding_dim]
        output, attn_weights = self.final_layer(output, mem,
                                                tgt_mask = tgt_mask, mem_mask = mem_mask,
                                                tgtkey_pad_mask = tgtkey_pad_mask,
                                                memkey_pad_mask = memkey_pad_mask
                                                )
        if self.norm:
            output = self.norm(output)
        return output, attn_weights

In [12]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model = 768, nheads = 12, dim_ffn = 3072, dropout = 0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nheads, dropout=dropout, device = device)
        self.multihead_attn = nn.MultiheadAttention(d_model, nheads, dropout=dropout, device = device)
        self.fc1 = Linear(d_model, dim_ffn)
        self.dropout = Dropout(dropout)
        self.fc2 = Linear(dim_ffn, d_model)
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.norm3 = LayerNorm(d_model)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)
        self.dropout3 = Dropout(dropout)
    
    def forward(self, tgt, memory, tgt_mask = None, memory_mask = None, tgt_key_padding_mask = None, memory_key_padding_mask = None):
        tgt2 = self.self_attn(tgt, tgt, tgt, attn_mask = tgt_mask, key_padding_mask = tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2 = self.multihead_attn(tgt, memory, memory, attn_mask = memory_mask, key_padding_mask = memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.fc2(self.dropout(nn.GELU('tanh')(self.fc1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt

In [13]:
class FinalDecoderLayer(nn.Module):
    def __init__(self, d_model = 768, nheads = 12, dim_ffn = 3072, dropout = 0.1):
        super(FinalDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nheads, dropout = dropout, device = device)
        self.dropout1 = nn.Dropout(p = dropout)
        self.norm1 = LayerNorm(d_model)
        self.multihead_attn = nn.MultiheadAttention(d_model, nheads, dropout = dropout, device = device)
        self.dropout2 = Dropout(p = dropout)
        self.norm2 = LayerNorm(d_model)
        self.fc1 = Linear(d_model, dim_ffn)
        self.dropout = Dropout(p = dropout)
        self.fc2 = Linear(dim_ffn, d_model)    
        self.dropout3 = Dropout(p = dropout)
        self.norm3 = LayerNorm(d_model)
        
    def forward(self, tgt, mem, tgt_mask = None, mem_mask = None, tgtkey_pad_mask = None,memkey_pad_mask = None):
        tgt2 = self.self_attn(tgt, tgt, tgt,
                              attn_mask=tgt_mask,
                              key_padding_mask=tgtkey_pad_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2, attn_dist = self.multihead_attn(tgt, mem, mem,
                                              attn_mask=mem_mask,
                                              key_padding_mask=memkey_pad_mask, 
                                              need_weights = True, average_attn_weights = False)
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.fc2(self.dropout(nn.GELU('tanh')(self.fc1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt, attn_dist

In [14]:
class Seq2seq(nn.Module):
    def __init__(self, src_vocab_size = 128, tgt_vocab_size = 128,
                 embedding_dim = 768, fcn_hidden_dim = 3072,
                 num_heads = 8, num_layers = 6, dropout = 0.1, src_tgt_mat= None):
        super(Seq2seq, self).__init__()
        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size
        self.embedding_dim = embedding_dim
        self.src2tgt = src_tgt_mat
        self.encoder = model.to(device)
        self.src_embed = copy.deepcopy(self.encoder.embeddings)
        self.tgt_embed = copy.deepcopy(self.encoder.embeddings)
        self.decoder_layer = ImportedTransformerDecoderLayer(embedding_dim, num_heads, fcn_hidden_dim, dropout, activation = 'gelu').to(device)
        self.decoder_final_layer = FinalDecoderLayer(embedding_dim, num_heads, fcn_hidden_dim, dropout).to(device)
        self.decoder = Decoder(self.decoder_layer, self.decoder_final_layer, num_layers).to(device)
        self.p_vocab = nn.Sequential(nn.Linear(self.embedding_dim*2, fcn_hidden_dim),
                                     nn.Linear(fcn_hidden_dim, self.tgt_vocab_size),
                                     nn.Softmax(dim = -1))
        self.src_mask = None
        self.mem_mask = None
        self.tgt_mask = None

        
    def return_boolean_masks(self, src_input_mask, tgt_input_mask):
        srckey_pad_mask = (src_input_mask == 0)
        tgtkey_pad_mask = (tgt_input_mask == 0)
        memkey_pad_mask = copy.deepcopy(srckey_pad_mask).to(device)
        return srckey_pad_mask, tgtkey_pad_mask, memkey_pad_mask
    
    def encode(self, src, srckey_pad_mask = None):
        encoder_out = self.encoder(src, attention_mask = srckey_pad_mask)
        mem = encoder_out['last_hidden_state'].transpose(0,1)
        return mem

    def decode(self, mem, src, tgt, memkey_pad_mask = None, tgtkey_pad_mask = None, has_mask = True):
        self.tgt_mask = _get_square_subsequent_mask(tgt.size(1)).to(device) # [T,T]
        tgt_embed = self.tgt_embed(tgt).transpose(0,1)
        decoder_out, attn = self.decoder(tgt_embed, mem, tgt_mask = self.tgt_mask, mem_mask = self.mem_mask,
                                         memkey_pad_mask = memkey_pad_mask, tgtkey_pad_mask = tgtkey_pad_mask
                                        ) # returns attention of only the last layer.
        # decoder_out of the shape [T,N,E]
        # attn of the shape [N,num_heads,T,S]
        A_t = attn.sum(dim = 1)
        context_vecs = torch.matmul(A_t, mem.transpose(0,1)).transpose(0,1)
        p_vocab = self.p_vocab(torch.cat((decoder_out, context_vecs), dim = -1)).transpose(0,1)
        return p_vocab, A_t, context_vecs, decoder_out, tgt_embed
        
    def forward(self, src, tgt, src_input_mask = None, tgt_input_mask = None, has_mask = True):
        srckey_pad_mask, tgtkey_pad_mask, memkey_pad_mask = self.return_boolean_masks(src_input_mask, tgt_input_mask)
        mem = self.encode(src, src_input_mask)
        out = self.decode(mem, src, tgt, memkey_pad_mask, tgtkey_pad_mask)
        return out

In [15]:
class PGN(nn.Module):
    def __init__(self, src_vocab_size = 128, tgt_vocab_size = 128,
                 embedding_dim = 768, fcn_hidden_dim = 3072,
                 num_heads = 8, num_layers = 6, dropout = 0.1, src_tgt_mat= None):
        super(PGN, self).__init__()
        self.src_vocab_size = src_vocab_size
        self.baseline = Seq2seq(src_vocab_size = src_vocab_size,
                             tgt_vocab_size = tgt_vocab_size,
                             num_layers = num_layers,
                             src_tgt_mat = src_tgt_mat,
                             embedding_dim = embedding_dim
                            ).to(device)
        self.p_gen = nn.Sequential(nn.Linear(embedding_dim*3,1),
                                   nn.Sigmoid()
                                  )
        self.conversion_matrix = src_tgt_mat
          
    def forward(self, src, tgt, src_input_mask = None, tgt_input_mask = None, has_mask = True):
        srckey_pad_mask, tgtkey_pad_mask, memkey_pad_mask = self.baseline.return_boolean_masks(src_input_mask, tgt_input_mask)
        mem = self.baseline.encode(src, src_input_mask)
        p_vocab, A_t, context_vecs, decoder_out, tgt_embed = self.baseline.decode(mem, src, tgt, memkey_pad_mask, tgtkey_pad_mask)
        total_states = torch.cat((context_vecs, decoder_out, tgt_embed), dim = -1)
        p_gen = self.p_gen(total_states)
        p_copy = 1 - p_gen
        oh = torch.zeros(src.size(0), src.size(1), self.src_vocab_size, device = device)
        oh_scattered = oh.scatter_(dim = -1, index = src.unsqueeze(-1), value = 1)
        p_copy_src_vocab = torch.matmul(A_t, oh)
        p_copy_tgt_vocab = torch.matmul(p_copy_src_vocab, self.conversion_matrix)
        p = torch.add(p_gen * p_vocab.transpose(0,1), p_copy * p_copy_tgt_vocab.transpose(0,1))
        return p.transpose(0,1)

In [16]:
vocab = tokenizer.get_vocab()
inv_vocab = {v:k for k,v in vocab.items()}

for key, value in tokenizer.special_tokens_map.items():
    print(f'{key} : {value} : {vocab[value]}')

bos_token : <s> : 0
eos_token : </s> : 2
unk_token : <unk> : 3
sep_token : </s> : 2
pad_token : <pad> : 1
cls_token : <s> : 0
mask_token : <mask> : 4


In [28]:
class ASansRoBERTa(nn.Module):
    def __init__(self, args):
        super(ASansRoBERTa, self).__init__()
        self.save_params = args
        self.max_len = args['max_len']
        self.world_size = args['gpus']
        self.train_batch_size = args['train_batch_size']
        self.eval_batch_size = args['eval_batch_size']
        self.epochs = args['epochs']
        self.label_smooth = args['label_smooth']
        self.tokenizer = tokenizer
        self.vocab = self.tokenizer.get_vocab()
        self.inv_vocab = {v:k for k,v in vocab.items()}
        self.vocab_size = len(self.vocab)
        self.unk_id = self.vocab['<unk>']
        self.sep_id = self.vocab['</s>']
        self.pad_id = self.vocab['<pad>']
        self.cls_id = self.vocab['<s>']
        self.max_src_len = args['src_seq_len']
        self.max_tgt_len = args['tgt_seq_len']
        self.num_layers = args['num_layers']
        self.optimizer_lr = args['optimizer_lr']
        self.optimizer_eps = args['optimizer_eps']
        self.optimizer_wd = args['optimizer_wd']
        self.iav = args['optimizer_initial_accumulator_value']
        self.scheduler_patience = args['scheduler_patience']
        self.embedding_dim = args['embedding_dim']
        self.scorer = Rouge()
        self.scheduler = None
        self.loss_fn = None
        self.train_data = self.load_train_data(args['train_file'], data_dir+'/train.pt', is_test = False)
        self.dev_data = self.load_train_data(args['dev_file'], data_dir+'/dev.pt', is_test = False)
        self.test_data = self.load_test_data(args['test_file'], data_dir+'/test.pt')
        self.conversion_matrix = self.conversion_matrix_()
        
#         self.model = Seq2seq(src_vocab_size = self.vocab_size,
#                              tgt_vocab_size = self.vocab_size,
#                              num_layers = self.num_layers,
#                              src_tgt_mat = self.conversion_matrix,
#                              embedding_dim = self.embedding_dim
#                             ).to(device)
        
        self.model = PGN(src_vocab_size = self.vocab_size,
                             tgt_vocab_size = self.vocab_size,
                             num_layers = self.num_layers,
                             src_tgt_mat = self.conversion_matrix,
                             embedding_dim = self.embedding_dim
                            ).to(device)
        
        print_model_status(model)
        
    def conversion_matrix_(self):
        conversion_matrix = torch.zeros(self.vocab_size, self.vocab_size)
        src_vocab_items = self.vocab.items()
        for src_token, src_id in src_vocab_items:
            tgt_id = self.vocab.get(src_token, self.unk_id)
            conversion_matrix[src_id][tgt_id] = 1
        return conversion_matrix.to(device)
    
    def get_params(self, param_name = None):
        self.save_params['train_data'] = self.train_data
        self.save_params['test_data'] = self.test_data
        self.save_params['val_data'] = self.dev_data
        self.save_params['model'] = self.model
        self.save_params['conv_mat'] = self.conversion_matrix
        if param_name is not None:
            return self.save_params[param_name]
        return self.save_params
    
    def load_train_data(self, file_path, loader_path, is_test = False):
        '''
            This function is for loading and saving the training data.
            From the training data we get the source ids and mask and the target ids, mask and ground truth values
            This saves the input data in a pt object and this is further used for the training.
        '''
        if os.path.exists(loader_path):
            print('Loading pre embedded data!')
            data = torch.load(loader_path)
        else:
            print('Creating Embeddings!')
            try:
                f = pd.read_csv(file_path)
            except:
                f = pd.read_csv(file_path, sep = "\t")
            src_data = f['text'].tolist()
            if not is_test:
                tgt_data = f['summary'].tolist()
            encoded_dict = self.tokenizer.batch_encode_plus(src_data,
                                                       add_special_tokens = True,
                                                       max_length = self.max_src_len,
                                                       padding = 'max_length',
                                                       return_attention_mask = True,
                                                       truncation = True,
                                                       return_tensors = 'pt')
            src_input_ids = encoded_dict['input_ids']
            src_attention_masks = encoded_dict['attention_mask']
            if not is_test:
#                 target_dict = self.tokenizer.batch_encode_plus(src_data,
#                                                               add_special_tokens = True,
#                                                               max_length = self.max_tgt_len,
#                                                               padding = 'max_length',
#                                                               return_attention_mask = True,
#                                                               truncation = True,
#                                                               return_tensors = 'pt')
                
                tgt_input_ids, tgt_ground_ids, tgt_attention_masks = [], [], []
                for text in tgt_data:
                    encoded_text = self.tokenizer(text)
                    tgt_ids, tgt_attention_mask = encoded_text['input_ids'], encoded_text['attention_mask']
                    if len(tgt_ids) > self.max_tgt_len:
                        print('len larger than maxlen')
                        tgt_ids = tgt_ids[:self.max_tgt_len-1]+[self.sep_id]
                        tgt_attention_mask = tgt_attention_mask[:self.max_tgt_len]
                    tgt_input, tgt_ground, tgt_mask = padding_trg(tgt_ids[:-1],
                                                                  tgt_ids[1:],
                                                                  tgt_attention_mask[:-1],
                                                                  self.max_tgt_len)
                    tgt_input_ids.append(tgt_input)
                    tgt_ground_ids.append(tgt_ground)
                    tgt_attention_masks.append(tgt_mask)
                data = {
                    'src_input_ids' : src_input_ids,
                    'src_attention_masks': src_attention_masks,
                    'trg_input_ids': torch.tensor(tgt_input_ids),
                    'trg_ground_ids': torch.tensor(tgt_ground_ids),
                    'trg_attention_masks': torch.tensor(tgt_attention_masks)
                }
            else:
                data = {
                    'src_input_ids': src_input_ids,
                    'src_attention_masks': src_attention_masks
                }
            torch.save(data, loader_path)
        return data
    
    def load_test_data(self, file_path, loader_path):
        '''Loads the test data and saves it as a tensor object for further usage.'''
        if os.path.exists(loader_path):
            data = torch.load(loader_path)
        else:
            src_data = []
            f = pd.read_csv(file_path)
            src_data = f['text'].tolist()
            encoded_dict = tokenizer.batch_encode_plus(src_data,
                                                       add_special_tokens=True,
                                                       max_length=self.max_src_len,
                                                       padding='max_length',
                                                       return_attention_mask=True,
                                                       truncation=True,
                                                       return_tensors='pt')
            src_input_ids = encoded_dict['input_ids']
            src_attention_masks = encoded_dict['attention_mask']
            data = {
                'src_input_ids': src_input_ids,
                'src_attention_masks': src_attention_masks
            }
            torch.save(data, loader_path)
        return data

    def my_dataloader(self, data_dict, batch_size, shuffle = True):
        '''Creates dataloaders for training and testing data.'''
        if 'trg_input_ids' in data_dict:
            dataset = TensorDataset(data_dict['src_input_ids'],
                                    data_dict['src_attention_masks'],
                                    data_dict["trg_input_ids"],
                                    data_dict["trg_ground_ids"],
                                    data_dict["trg_attention_masks"])
        else:
            dataset = TensorDataset(data_dict["src_input_ids"],
                                    data_dict["src_attention_masks"])

        if shuffle:
            sampler = RandomSampler(dataset)
            dataset_loader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
        else:
            dataset_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return dataset_loader
    
    
    def get_loss(self, predict, target, label_smooth = 0.1):
        predict = predict.contiguous().view(-1, self.vocab_size)
        target = target.contiguous().view(-1, 1)
        non_pad_mask = target.ne(self.pad_id)
        nll_loss = -predict.gather(dim = -1, index = target)[non_pad_mask].mean()
        smooth_loss = -predict.sum(dim = -1, keepdim = True)[non_pad_mask].mean()
        smooth_loss = smooth_loss/self.vocab_size
        loss = ((1. - label_smooth) * nll_loss) + (label_smooth * smooth_loss)
        return loss
    
    def write_data_to_file(self, data):
        preds = data.topk(1)[1].squeeze()
        write_to_file(preds, '/kaggle/working/predictions.txt')
    
    def get_accuracy(self, predict, target, mode, batch_size = 2, mask = None):
        preds = predict.topk(1)[1].squeeze()
        score = 0
        score_prev = 0
        if mode in ['test', 'train', 'valid']:
            predicted = decode(preds, back = True)
            ground_truths = decode(target, back = True)
            for i in range(len(ground_truths)):
                score_prev = self.scorer.get_scores(predicted[i], ground_truths[i], avg = True)
                score += score_prev['rouge-1']['f']
            return score
        if mode in ['train', 'valid']:
            accuracy = 0
            if mask == None or batch_size == None:
                print('Not enough information!')
                return 0.0
            preds = preds * mask
            for i in range(batch_size):
                non_zero_mask = preds[i].ne(0.0)
                total_value = non_zero_mask.sum().item()
                correct_pred = torch.eq(preds[i][non_zero_mask], target[i][non_zero_mask]).sum().item()
                accuracy += correct_pred * (100.0 / total_value)
            return accuracy/batch_size
    
    def validation(self, epoch):
        dev_loader = self.my_dataloader(self.dev_data, self.eval_batch_size)
        self.model.eval()
        running_loss = 0
        accuracy = 0
        i = 0
        with torch.no_grad():
            for batch in dev_loader:
                src_input_ids, src_input_mask = batch[0].to(device), batch[1].to(device)
                tgt_input_ids, tgt_input_mask = batch[2].to(device), batch[4].to(device)
                tgt_ground_ids = batch[3].to(device)
                out = self.model(src_input_ids, tgt_input_ids, src_input_mask, tgt_input_mask)
                loss = self.loss_fn(torch.log(out.transpose(1,2)), tgt_ground_ids)
                #loss = self.get_loss(out, tgt_ground_ids)
                running_loss += loss.item()
                accuracy += self.get_accuracy(out, tgt_ground_ids, 'valid', self.eval_batch_size, mask = tgt_input_mask)
                self.write_data_to_file(out)
                self.scheduler.step(loss)
                del src_input_ids
                del src_input_mask
                del tgt_input_ids
                del tgt_input_mask
                del tgt_ground_ids
                i += 1
        final_loss = running_loss / (i+1)
        accuracy = accuracy / len(dev_loader)
        print(f"Validation Epoch : {epoch}, Average Validation Loss : {final_loss:.4f}, Validation Accuracy : {accuracy:.2f}%")
        #print(f"Validation Epoch : {epoch}, Average Validation Loss : {final_loss:.4f}")
        return final_loss
    
    def train(self, pgn_model_path = None, baseline_checkpoint_path = None, pgn_checkpoint_path = None , flag = False):
        if pgn_model_path:
            print('Restoring PGN Model!')
            model = torch.load(pgn_model_path)
            pgn_checkpoint = torch.load(pgn_checkpoint_path)
            self.model = model
            
        if pgn_checkpoint_path and baseline_checkpoint_path:
            print('Restoring both Models from checkpoint!')
            pgn_checkpoint = torch.load(pgn_checkpoint_path, map_location = 'cpu')
            baseline_checkpoint = torch.load(baseline_checkpoint_path, map_location = 'cpu')
            self.model.load_state_dict(fix_state_dict(checkpoint['model_state_dict']), strict = False)
            self.model.baseline.load_state_dict(fix_state_dict(checkpoint['model_state_dict']), strict = False)
        
        if pgn_checkpoint_path:
            print('Restoring PGN Model from checkpoint!')
            pgn_checkpoint = torch.load(pgn_checkpoint_path, map_location = 'cpu')
            self.model.load_state_dict(fix_state_dict(checkpoint['model_state_dict']), strict = False)
            
        if baseline_checkpoint_path:
            print('Restoring Baseline Model from checkpoint!')
            baseline_checkpoint = torch.load(baseline_checkpoint_path, map_location = 'cpu')
            self.model.baseline.load_state_dict(fix_state_dict(checkpoint['model_state_dict']), strict = False)
        
        train_loader = self.my_dataloader(self.train_data, self.train_batch_size)
        start_epoch = 0
        epochs = self.epochs
        if pgn_model_path or pgn_checkpoint_path:
            print('Restoring Epochs!')
            start_epoch = pgn_checkpoint['epoch']+1
            epochs = epochs + start_epoch
        # used to freeze encoder params and train.
        optimizer_grouped_params = [
            {'params': self.model.baseline.encoder.parameters(), 'lr': 1e-5, 'weight_decay': self.optimizer_wd}, # roberta encoder
            {'params': self.model.baseline.tgt_embed.parameters(), 'lr': 1e-5, 'weight_decay': 1e-3}, # model embeddings
            {'params': self.model.baseline.decoder.parameters(), 'lr': self.optimizer_lr}, # roberta decoder
            {'params': self.model.baseline.p_vocab.parameters(), 'lr': self.optimizer_lr}, # p_vocab
            {'params': self.model.p_gen.parameters(), 'lr': self.optimizer_lr}, # p_gen
            
            #{'params': self.model.parameters(), 'lr': self.optimizer_lr}
        ]
        loss_fn = nn.NLLLoss(reduction = 'mean', ignore_index = self.pad_id) # ignore_index = self.pad_id
        #loss_fn = nn.CrossEntropyLoss(reduction = 'mean', ignore_index = self.pad_id, label_smoothing = 0.1)
        self.loss_fn = loss_fn
        optimizer = AdamW(optimizer_grouped_params, lr = self.optimizer_lr, eps = self.optimizer_eps, weight_decay = 1e-3)
        total_steps = len(train_loader) * epochs
        #scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = total_steps/10, num_training_steps = total_steps)
        scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps = total_steps/10)
        self.scheduler = scheduler
        
        if flag == True:
            print('Learning Rate and Scheduler Changed!')
        else:
            if pgn_model_path or pgn_checkpoint_path:
                print('Optimizer and Scheduler state_dict restored!')
                optimizer.load_state_dict(pgn_checkpoint['optimizer_state_dict'])
                self.scheduler.load_state_dict(pgn_checkpoint['scheduler_state_dict'])

        is_best = False
        curr_valid_loss = 0
        plot_train_loss = []
        plot_valid_loss = []
        if pgn_checkpoint_path or pgn_model_path:
            plot_train_loss = pgn_checkpoint['plot_train_loss']
            plot_valid_loss = pgn_checkpoint['plot_valid_loss']
        best_valid_loss = float('inf')
        epochs_no_improve = 0
        total_steps = 0
        stime = time.ctime()
        start_time = time.monotonic()
        print(f'Training started at : {stime}')
        for epoch in range(start_epoch, epochs):
            self.model.train()
            print(f'Epoch / Total Epochs : {epoch} / {epochs}')
            running_loss = 0.0
            accuracy = 0.0
            for batch in tqdm(train_loader):
                src_input_ids, src_input_mask = batch[0].to(device), batch[1].to(device)
                tgt_input_ids, tgt_input_mask = batch[2].to(device), batch[4].to(device)
                tgt_ground_ids = batch[3].to(device)
                optimizer.zero_grad()
                out = self.model(src_input_ids, tgt_input_ids, src_input_mask, tgt_input_mask) # out of shape (batch_size, max_len, tgt_vocab_dist)
                del src_input_ids
                del src_input_mask
                del tgt_input_ids
                loss = self.loss_fn(torch.log(out.transpose(1,2)), tgt_ground_ids)
                #loss = self.get_loss(out, tgt_ground_ids)
                accuracy +=  self.get_accuracy(out, tgt_ground_ids, 'train', self.train_batch_size, tgt_input_mask)
                self.write_data_to_file(out)
                del tgt_input_mask
                del tgt_ground_ids
                loss.backward()
                optimizer.step()
                self.scheduler.step(loss)                
                running_loss += loss.item()
                total_steps += 1
            plot_train_loss.append(loss.item())
            if total_steps % 100 == 0 or epoch % 1 == 0:
                print(f'Train Epoch : {epoch}, Average Training Loss : {running_loss/len(train_loader):.4f}, Train Accuracy : {accuracy/len(train_loader):.2f}%')
                #print(f'Train Epoch : {epoch}, Average Training Loss : {running_loss/len(train_loader):.4f}')
            if epoch % 1 == 0:
                epochs_no_improve += 1
                curr_valid_loss = self.validation(epoch)
                plot_valid_loss.append(curr_valid_loss)
                if curr_valid_loss < best_valid_loss:
                    print('New Best Loss!')
                    is_best = True
                    best_valid_loss = curr_valid_loss
                    best_valid_epoch = epoch
                    epochs_no_improve = 0
                    #torch.save(self.model, save_dir+f'model.pt')
                    torch.save({
                        'train_loss' : running_loss,
                        'valid_loss' : best_valid_loss,
                        'plot_train_loss' : plot_train_loss,
                        'plot_valid_loss' : plot_valid_loss,
                        'epoch' : epoch,
                        'optimizer_state_dict' : optimizer.state_dict(),
                        'scheduler_state_dict' : self.scheduler.state_dict(),
                        'model_state_dict' : self.model.state_dict()}, save_dir+f'pgn_model_checkpoint.pt')
                if epochs_no_improve > 8:
                    print('No change in validation loss. Training not useful! Stopping Training.')
                    break
        etime = time.ctime()
        end_time = time.monotonic()
        print(f'Training ends at : {etime}. Took almost {(end_time - start_time)/60} mins.')

In [29]:
params = {'train_file' : data_dir+'/Data/newtrain.csv',
          'dev_file' : data_dir+'/Data/newdev.csv',
          'test_file' : data_dir+'/Data/newtest.csv',
          'max_len' : 512,
          'gpus' : 1,
          'train_batch_size' : 2,
          'eval_batch_size' : 2,
          'epochs' : 50,
          'label_smooth' : 0.1,
          'src_seq_len' : 512,
          'tgt_seq_len' : 64,
          'num_layers' : 6,
          'optimizer_lr' : 1e-2,
          'optimizer_wd' : 1e-2,
          'optimizer_eps' : 1e-8,
          'optimizer_initial_accumulator_value' : 0.1,
          'scheduler_patience' : 1,
          'embedding_dim' : 768
         }

In [30]:
free_gpu_cache(stats = True)

In [31]:
asans = ASansRoBERTa(params).to(device)
gpu_mem()
#print_model_status(asans)

Loading pre embedded data!
Loading pre embedded data!
False


In [21]:
# out = asans.train(pre_checkpoint_path = data_dir+'/', flag = True)
# out = asans.train(baseline_checkpoint_path = data_dir+'/model_checkpoint9_671.pt', pgn_checkpoint_path = save_dir+'pgn_model_checkpoint.pt', flag = True)

<h1><a href = 'pgn_model_checkpoint.pt'>Download</a></h1>

In [32]:
def check(ckp_path):
    score = 0
    #free_gpu_cache(stats = True)
    #data = asans.load_train_data(data_dir+'data.csv', save_dir+'new_data.pt', is_test = False)
    batch = next(iter(asans.my_dataloader(asans.load_train_data(data_dir+'data.csv', save_dir+'new_data.pt', is_test = False), 10)))
    srci, srcm, tgti, tgtg, tgtm = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device), batch[4].to(device)
    checkpoint = torch.load(ckp_path, map_location = 'cpu')
    #print(checkpoint['epoch'])
    mdl = asans.get_params('model')
    mdl.load_state_dict(checkpoint['model_state_dict'], strict = False);
    out = mdl(srci, tgti, srcm, tgtm)
    x = out.topk(1)[1].squeeze()
    ground = decode(tgtg, back = True)
    preds = decode(x, back = True)
    orig = decode(srci, back = True)
    for i in range(len(ground)):
        score_prev = asans.scorer.get_scores(preds[i], ground[i], avg = True)
        score += score_prev['rouge-1']['f']
    df = pd.DataFrame(columns = ['original_text', 'predicted_summary', 'ground_summary'])
    for i in range(len(ground)):
        df.loc[len(df)] = {'original_text': orig[i],'predicted_summary': preds[i], 'ground_summary': ground[i]}
    print(score)
    return df
    #df.to_csv('/kaggle/working/preds.csv')
    #return download('preds.csv')

In [33]:
df = check(ckp_path = save_dir+'pgn_model_checkpoint.pt')
df.to_csv('roberta_pgn_predictions.csv')
df

Loading pre embedded data!
0.0


,original_text,predicted_summary,ground_summary
0,<s>प्राचीनयुगात् अस्माकं समाजे स्त्रीणां विशिष...,व्रारात्र्र्र्यारास्र्श्य्याता वित्त्र्स्यार्स...,प्राचीनयुगात् अस्माकं समाजे स्त्रीणां विशिष्टं...
1,<s>वर्षाकाल सुखकरः भवति । अस्मिन् काले विशेषतय...,क्रान्र््र्</s>्त्</s>्शात्त्न्याार्य्य्त</s>्...,वर्षाकाल सुखकरः भवति । ते कृषिकार्य कुर्वन्ति...
2,<s>अध्ययनस्य सर्वथा परिवर्तन जातम् तथापि यावत्...,व्य्या्य्नित्य्ान्यान्तान्यार्य्</s>्र्य्निरार...,अध्ययनस्य सर्वथा परिवर्तन जातम् तथापि यावत् छा...
3,<s>अयं अस्माकं विद्यालयः अस्ति । अस्य भवनानि भ...,व्स्यार्सित्यारा</s>्य्त््या्यार्रियार्््र्या<...,अयं अस्माकं विद्यालयः अस्ति । अस्य प्रधानाध्या...
4,<s>यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि...,सियाितातिन्त्द्यिन्त्न्यास्स्तिय्न्त्य्त्य्यान...,यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि भव...
5,<s>शासनमनु अनुशासनम् अर्थात शासनेन निर्मितानि ...,व्र्र्र्र्र्यार््र्श्ित्यात्न्तित्त्न्य्</s>्त...,शासनमनु अनुशासनम् अर्थात शासनेन निर्मितानि निय...
6,<s>सताम् आचारः सदाचारः कथ्यते । सज्जनाः यानि क...,व्रायान् क्नान्</s>ायिन</s>्या्य्तानित्त्य्यान...,सताम् आचारः सदाचारः कथ्यते । ऋषयश्च वदन्ति यान...
7,<s>अस्माकं देशे सर्वासु नदीषु गंगा अतिश्रेष्ठा...,कारारासिश्श्यार्रात्र्स्रित्य्श्यिर्यार्र्त्य्...,अस्माकं देशे सर्वासु नदीषु गंगा अतिश्रेष्ठा प्...
8,<s>अस्माकं देशः भारतवर्षम् अस्ति । अयं हि हिमा...,स्रारा पिश् पार्य्र्र्त््राा पा््र्््त्र््ार्य...,"अस्माकं देशः भारतवर्षम् अस्ति । अत्र गंगा, यमु..."
9,<s>यस्मिन् देशे वयं जन्मधारणं कुर्मः स हि अस्म...,स्रातायिश्श्स्य्रा पार्या</s>्ात्यार्स्श्</s>्...,यस्मिन् देशे वयं जन्मधारणं कुर्मः स हि अस्माकं...
